In [1]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1TyXV0KPUbbNeiPeMVaMGd1MGeO5zCfx4' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1TyXV0KPUbbNeiPeMVaMGd1MGeO5zCfx4" -O EEG_data.csv && rm -rf /tmp/cookies.txt

--2023-12-07 21:45:14--  https://docs.google.com/uc?export=download&confirm=t&id=1TyXV0KPUbbNeiPeMVaMGd1MGeO5zCfx4
Resolving docs.google.com (docs.google.com)... 209.85.147.113, 209.85.147.138, 209.85.147.102, ...
Connecting to docs.google.com (docs.google.com)|209.85.147.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-00-5g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/1qoph66l66bo6pvtk5c5hh2jbrljjobb/1701985500000/15697722048597459399/*/1TyXV0KPUbbNeiPeMVaMGd1MGeO5zCfx4?e=download&uuid=02f30622-66e5-41ed-a52b-f4e3b11d3ed1 [following]
--2023-12-07 21:45:14--  https://doc-00-5g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/1qoph66l66bo6pvtk5c5hh2jbrljjobb/1701985500000/15697722048597459399/*/1TyXV0KPUbbNeiPeMVaMGd1MGeO5zCfx4?e=download&uuid=02f30622-66e5-41ed-a52b-f4e3b11d3ed1
Resolving doc-00-5g-docs.googleusercontent.com (doc-00-5g-docs.googleusercontent.com)... 64.233.182.132, 

In [2]:
import pandas as pd

data = pd.read_csv('EEG_data.csv', index_col='Unnamed: 0')

data.shape

(2100000, 30)

In [3]:
data.head()

,Channel 1 - P4,Channel 2 - O2,Channel 3 - P8,Channel 4 - T8,Channel 5 - C4,Channel 6 - Cz,Channel 7 - Fz,Channel 8 - F4,Channel 9 - Fp2,Channel 10 - F8,...,EC,EO,ET,F,H,M,R,S,gender,porn_addiction
0,7.123764,5.649603,3.406298,-2.418010,13.527913,8.277149,1.529848,7.362843,1.342155,1.428773,...,1,0,0,0,0,0,0,0,1,1
1,3.576627,3.938701,3.019094,-0.579628,5.340958,6.342993,-2.007120,1.416022,-0.951962,0.987006,...,1,0,0,0,0,0,0,0,1,1
2,0.852858,1.792533,1.908249,1.051835,1.018308,3.018750,-3.727451,-2.563925,-2.922473,0.136649,...,1,0,0,0,0,0,0,0,1,1
3,-0.400484,0.084309,0.814120,1.609101,-0.364434,-0.421646,-3.760435,-3.945382,-3.583919,-0.625634,...,1,0,0,0,0,0,0,0,1,1
4,0.129493,-0.513726,0.226075,0.895274,0.268336,-2.704263,-2.235362,-2.555360,-2.894571,-0.994642,...,1,0,0,0,0,0,0,0,1,1


In [4]:
data.iloc[100000]

Channel 1 - P4     -1.913375
Channel 2 - O2      0.093344
Channel 3 - P8      0.573907
Channel 4 - T8      0.201486
Channel 5 - C4      1.231755
Channel 6 - Cz      1.146246
Channel 7 - Fz      0.553911
Channel 8 - F4     -3.676762
Channel 9 - Fp2    -2.503975
Channel 10 - F8     0.763982
Channel 11 - Fp1    2.081112
Channel 12 - F7     1.272596
Channel 13 - F3     0.144714
Channel 14 - C3     0.674265
Channel 15 - T7     1.421907
Channel 16 - P7     0.801090
Channel 17 - P3    -0.410703
Channel 18 - O1    -0.051313
Channel 19 - Pz    -2.745100
C                   0.000000
EC                  0.000000
EO                  1.000000
ET                  0.000000
F                   0.000000
H                   0.000000
M                   0.000000
R                   0.000000
S                   0.000000
gender              1.000000
porn_addiction      1.000000
Name: 100000, dtype: float64

In [5]:
class Leave_one_out():
    
    def __init__(self, dataset, model, subsets_no, prediction_label):
        self.dataset = data
        self.patient_subjects = []
        self.model = model
        self.patient_no = subsets_no
        self.prediction_label = prediction_label
        self.data_len = len(self.dataset)
        self.num_rows_per_df = self.data_len // self.patient_no
        
        for i in range(self.patient_no):
            start_index = i * self.num_rows_per_df
            end_index = (i + 1) * self.num_rows_per_df
            subject = dataset.iloc[start_index:end_index]
            self.patient_subjects.append(subject)
        
    def __split(self, selected_index):
        test_data = self.patient_subjects[selected_index]
        train_data = [self.patient_subjects[i] for i in range(self.patient_no) if i != selected_index]
        train_labels = [i for i in range(self.patient_no) if i != selected_index]
        train_data = pd.concat(train_data, ignore_index=True)
        print(f"Leaving patient {selected_index} out, training on patients {', '.join(map(str, train_labels))}")
        X_train = train_data.drop(self.prediction_label, axis = 1)
        y_train = train_data[self.prediction_label]
        X_test = test_data.drop(self.prediction_label, axis = 1)
        y_test = test_data[self.prediction_label]
        return X_train, y_train, X_test, y_test
        
    def train(self, epoch, batch_size, validation_split = 0, callbacks=[]):
        results = []
        for selected_index in range(self.patient_no):
            X_train, y_train, X_test, y_test = self.__split(selected_index)
            model = self.model()
            print('Training....')
            train_history = model.fit(X_train, y_train, epochs = epoch, batch_size = batch_size, callbacks=callbacks, validation_split = validation_split)
            print('Test....')
            test_accuracy = model.evaluate(X_test, y_test, batch_size = batch_size)
            print()
            results.append((train_history, test_accuracy))
        return results

In [6]:
import tensorflow as tf

def build_model(num_classes=1):
    inputs = tf.keras.Input(shape=(29,))
    x = tf.keras.layers.Reshape((-1, 29))(inputs) 
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True))(x)
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(x)
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='sigmoid')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [7]:
test = Leave_one_out(data, build_model, 14, 'porn_addiction')

In [8]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='loss', patience=5, verbose=1)

models = test.train(epoch = 5, batch_size = 1250, callbacks = [early_stopping])

Leaving patient 0 out, training on patients 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13
Training....
Epoch 1/5
1560/1560 [==============================] - 22s 7ms/step - loss: 0.3462 - accuracy: 0.8436
Epoch 2/5
1560/1560 [==============================] - 11s 7ms/step - loss: 0.2434 - accuracy: 0.8962
Epoch 3/5
1560/1560 [==============================] - 11s 7ms/step - loss: 0.2054 - accuracy: 0.9139
Epoch 4/5
1560/1560 [==============================] - 11s 7ms/step - loss: 0.1829 - accuracy: 0.9239
Epoch 5/5
1560/1560 [==============================] - 11s 7ms/step - loss: 0.1667 - accuracy: 0.9313
Test....
120/120 [==============================] - 2s 3ms/step - loss: 2.1893 - accuracy: 0.3909

Leaving patient 1 out, training on patients 0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13
Training....
Epoch 1/5
1560/1560 [==============================] - 17s 7ms/step - loss: 0.3757 - accuracy: 0.8263
Epoch 2/5
1560/1560 [==============================] - 11s 7ms/step - loss: 0.2587 - accura